In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix
import re
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('wordnet')

# breather

import string 
import re
import codecs
import seaborn as sns
from sklearn import feature_extraction
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import pipeline
from sklearn import metrics
import itertools

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
import tensorflow as tf

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

In [ ]:
def read_conll(in_file, lowercase=False, max_example=None):
    csv_list = []
    with open(in_file) as f:
        for line in f.readlines():
            csv=[]
            sp = line.strip().split('\t')
            if len(sp)>1:
               csv.append(sp[0])
               csv.append(sp[1])
            csv_list.append(csv)
    return pd.DataFrame(csv_list)

In [ ]:
lid_train=read_conll("/content/drive/MyDrive/lid_spaeng/lid_spaeng/train.conll") # lid only data
temp=lid_train
# combined data of lid+kaggle
lid_dev=read_conll("/content/drive/MyDrive/lid_spaeng/lid_spaeng/dev.conll") # lid dev data
#lid_dev_bin=pd.read_csv("/content/drive/MyDrive/ML JCOMP SEM5/1A_English_actual_labels.csv") # lid binary dev labels
#lid_dev_mul=pd.read_csv("/content/drive/MyDrive/ML JCOMP SEM5/1B_English_actual_labels.csv") #lid multiclass dev labels

In [ ]:
lid_train.tail(10)

,0,1
295271,!!!!!!!,other
295272,None,None
295273,None,None
295274,@ThatGirlYahs,other
295275,callata,lang2
295276,borrega,lang2
295277,pelos,lang2
295278,necios,lang2
295279,!!!!!,other
295280,None,None


In [ ]:
lid_train[1] = np.where(lid_train[1] == "fw", "other",lid_train[1])
lid_train[1] = np.where(lid_train[1] == "mixed", "other",lid_train[1])
lid_train[1] = np.where(lid_train[1] == "ambiguous", "other",lid_train[1])
lid_train[1] = np.where(lid_train[1] == "unk", "other",lid_train[1])

lid_dev[1] = np.where(lid_dev[1] == "fw", "other",lid_dev[1])
lid_dev[1] = np.where(lid_dev[1] == "mixed", "other",lid_dev[1])
lid_dev[1] = np.where(lid_dev[1] == "ambiguous", "other",lid_dev[1])
lid_dev[1] = np.where(lid_dev[1] == "unk", "other",lid_dev[1])

In [ ]:
lid_train[1].value_counts()*100/lid_train.shape[0]

lang2    38.264568
lang1    27.240832
other    18.493909
ne        1.756632
Name: 1, dtype: float64

In [ ]:
lid_train.columns=["word","label"]
lid_dev.columns=["word","label"]
lid_train=lid_train[(lid_train["word"].isnull()==False)]

In [ ]:
lid_dev=lid_dev[(lid_dev["word"].isnull()==False)]

In [ ]:
lid_train.describe()

,word,label
count,253221,253221
unique,39193,4
top,.,lang2
freq,5663,112988


In [ ]:
lid_train.head()

,word,label
1,11:11,other
2,.....,other
3,make,lang1
4,a,lang1
5,wish,lang1


In [ ]:
lid_dev.head()

,word,label
1,@_easanti,other
2,@mememecaigo,other
3,#todossomoscarlosperez,other
4,#hashtaglargo,lang2
7,I,lang1


In [ ]:
lemmatizer = WordNetLemmatizer()
'''stop_words= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","not"])'''

'stop_words= set([\'br\', \'the\', \'i\', \'me\', \'my\', \'myself\', \'we\', \'our\', \'ours\', \'ourselves\', \'you\', "you\'re", "you\'ve",            "you\'ll", "you\'d", \'your\', \'yours\', \'yourself\', \'yourselves\', \'he\', \'him\', \'his\', \'himself\',             \'she\', "she\'s", \'her\', \'hers\', \'herself\', \'it\', "it\'s", \'its\', \'itself\', \'they\', \'them\', \'their\',            \'theirs\', \'themselves\', \'what\', \'which\', \'who\', \'whom\', \'this\', \'that\', "that\'ll", \'these\', \'those\',             \'am\', \'is\', \'are\', \'was\', \'were\', \'be\', \'been\', \'being\', \'have\', \'has\', \'had\', \'having\', \'do\', \'does\',             \'did\', \'doing\', \'a\', \'an\', \'the\', \'and\', \'but\', \'if\', \'or\', \'because\', \'as\', \'until\', \'while\', \'of\',             \'at\', \'by\', \'for\', \'with\', \'about\', \'against\', \'between\', \'into\', \'through\', \'during\', \'before\', \'after\',            \'above\', \'below\', \'to\', \'f

In [ ]:
def preprocess(tweets):
  
  tweets.encode('ascii', 'ignore').decode('ascii') #remove emojis
  tweets=tweets.lower()                   #convert to lower case
  tweets = re.sub(r"http\S+", " ", tweets) #remove urls
  l=tweets.split(" ")
  for n,i in enumerate(l):
    if '@' in i:
      l[n]=""
  
  tweets=" ".join(l)
  tweets= re.sub('[^A-Za-z]+',' ',tweets) #remove bad character
  #tweets = [word for word in tweets.split(" ") if not word in stop_words] # removing stop words
  tweets= [lemmatizer.lemmatize(token, "v") for token in tweets] #Lemmatization
  tweets=" ".join(tweets) 
  return tweets 
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

In [ ]:
lid_train["word"]=lid_train["word"].apply(lambda x:decontract(x))
lid_train["word"]=lid_train["word"].apply(lambda x:preprocess(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
lid_dev["word"]=lid_dev["word"].apply(lambda x:decontract(x))
lid_dev["word"]=lid_dev["word"].apply(lambda x:preprocess(x))

In [ ]:
len(lid_dev)

40391

In [ ]:
lid_train.head()

,word,label
1,,other
2,,other
3,m a k e,lang1
4,a,lang1
5,w i s h,lang1


In [ ]:
lid_dev.head()

,word,label
1,,other
2,,other
3,t o d o s s o m o s c a r l o s p e r e z,other
4,h a s h t a g l a r g o,lang2
7,i,lang1


In [ ]:
lid_dev.at[2,'word']

''

In [ ]:
df = lid_train[lid_train.word != ' ']

In [ ]:
df_dev = lid_dev[lid_dev.word != ' ']

In [ ]:
df_dev = df_dev[df_dev.word != '']

In [ ]:
df.head()

,word,label
3,m a k e,lang1
4,a,lang1
5,w i s h,lang1
7,n i g h t,lang1
8,n i g h t,lang1


In [ ]:
df_dev.head()

,word,label
3,t o d o s s o m o s c a r l o s p e r e z,other
4,h a s h t a g l a r g o,lang2
7,i,lang1
8,d o,lang1
9,n o t,lang1


In [ ]:
df.iat[1,0]

'a'

In [ ]:
df = df[df.word != '']

In [ ]:
len(df)

199963

In [ ]:
len(lid_train)

253221

In [ ]:
len(lid_dev)

40391

In [ ]:
len(df_dev)

32758

In [ ]:
le=LabelEncoder()
#for multiclass data

lid_train["label"] = le.fit_transform(lid_train["label"])
lid_dev["label"] = le.fit_transform(lid_dev["label"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
df["label"] = le.fit_transform(df["label"])
df_dev["label"] = le.fit_transform(df_dev["label"])

In [ ]:
lid_train.head()

,word,label
1,,3
2,,3
3,m a k e,0
4,a,0
5,w i s h,0


In [ ]:
df.head()

,word,label
3,m a k e,0
4,a,0
5,w i s h,0
7,n i g h t,0
8,n i g h t,0


In [ ]:
lid_dev.head()

,word,label
1,,3
2,,3
3,t o d o s s o m o s c a r l o s p e r e z,3
4,h a s h t a g l a r g o,1
7,i,0


In [ ]:
df_dev.head()

,word,label
3,t o d o s s o m o s c a r l o s p e r e z,3
4,h a s h t a g l a r g o,1
7,i,0
8,d o,0
9,n o t,0


In [ ]:
X_train = df['word']
X_train = X_train.to_numpy()
len(X_train)

199963

In [ ]:
y_train = df['label']
y_train = y_train.to_numpy()
len(y_train)

199963

In [ ]:
X_test = df_dev['word']
X_test = X_test.to_numpy()
len(X_test)

32758

In [ ]:
newX_test = lid_dev['word']
newX_test = newX_test.to_numpy()
len(newX_test)

40391

In [ ]:
newy_test = lid_dev['label']
newy_test = newy_test.to_numpy()
len(newy_test)

40391

In [ ]:
X_train

array(['m a k e', 'a', 'w i s h', ..., 'b o r r e g a', 'p e l o s',
       'n e c i o s'], dtype=object)

In [ ]:
y_train

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
# ----- TILL HERE LIES THE PREPROCESSING WITHOUT ANY VECTORISATION -----

In [ ]:
vectorizer = feature_extraction.text.TfidfVectorizer(ngram_range=(1,3), analyzer='char')

In [ ]:
#train_vec = vectorizer.fit(X_train,y_train)

In [ ]:
#xtrain = vectorizer.fit_transform(X_train)

In [ ]:
#xtest = vectorizer.fit_transform(X_train)

In [ ]:
X_train.shape

(199963,)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
pipe_mnb = pipeline.Pipeline([
    ('vectorizer', vectorizer),
    ('clf', MultinomialNB())
])

In [ ]:
pipe_mnb.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
                ('clf', MultinomialNB())])

In [ ]:
pipe_lr = pipeline.Pipeline([
    ('vectorizer', vectorizer),
    ('lr_clf', LogisticRegression())
])

In [ ]:
pipe_lr.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
                ('lr_clf', LogisticRegression())])

In [ ]:
pipe_rf = pipeline.Pipeline([
    ('vectorizer', vectorizer),
    ('rf_clf', RandomForestClassifier())
])

In [ ]:
pipe_rf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
                ('rf_clf', RandomForestClassifier())])

In [ ]:
pipe_knn = pipeline.Pipeline([
    ('vectorizer', vectorizer),
    ('knn_clf', KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2))
])

In [ ]:
pipe_knn.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
                ('knn_clf', KNeighborsClassifier())])

In [ ]:
knn_predicted = pipe_knn.predict(X_test)

In [ ]:
knn_acc = (metrics.accuracy_score(y_test, knn_predicted))*100
print('The KNN model has:',knn_acc,'% accuracy')

The KNN model has: 94.01978142743758 % accuracy


In [ ]:
print(metrics.classification_report(y_test,knn_predicted))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95     16710
           1       0.94      0.96      0.95     14955
           2       0.79      0.47      0.59       814
           3       0.84      0.59      0.70       279

    accuracy                           0.94     32758
   macro avg       0.88      0.74      0.80     32758
weighted avg       0.94      0.94      0.94     32758



In [ ]:
lr_predicted = pipe_lr.predict(X_test)

In [ ]:
lr_acc = (metrics.accuracy_score(y_test, lr_predicted))*100
print('The logistic regression has:',lr_acc,'% accuracy')

The logistic regression has: 89.85591305940534 % accuracy


In [ ]:
print(metrics.classification_report(y_test,lr_predicted))

              precision    recall  f1-score   support

           0       0.93      0.90      0.91     16710
           1       0.87      0.95      0.91     14955
           2       0.63      0.09      0.16       814
           3       0.95      0.48      0.64       279

    accuracy                           0.90     32758
   macro avg       0.85      0.61      0.66     32758
weighted avg       0.90      0.90      0.89     32758



In [ ]:
mnb_predicted = pipe_mnb.predict(X_test)

In [ ]:
mnb_acc = (metrics.accuracy_score(y_test, mnb_predicted))*100
print('The MultinomialNB has :',mnb_acc,'% accuracy')

The MultinomialNB has : 78.34422125892911 % accuracy


In [ ]:
print(metrics.classification_report(y_test,mnb_predicted))

              precision    recall  f1-score   support

           0       0.89      0.71      0.79     16710
           1       0.71      0.92      0.80     14955
           2       0.33      0.01      0.03       814
           3       0.77      0.15      0.26       279

    accuracy                           0.78     32758
   macro avg       0.68      0.45      0.47     32758
weighted avg       0.79      0.78      0.77     32758



In [ ]:
rf_predicted = pipe_rf.predict(X_test)

In [ ]:
rf_acc = (metrics.accuracy_score(y_test, rf_predicted))*100
print('The RandomForest has :',rf_acc,'% accuracy')

The RandomForest has : 95.08517003480065 % accuracy


In [ ]:
print(metrics.classification_report(y_test,rf_predicted))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96     16710
           1       0.93      0.98      0.96     14955
           2       0.86      0.54      0.67       814
           3       0.86      0.61      0.71       279

    accuracy                           0.95     32758
   macro avg       0.91      0.77      0.82     32758
weighted avg       0.95      0.95      0.95     32758



In [ ]:
'''import tensorflow as tf

# Parameters
embedding_dim = 16
lstm_dim = 32
dense_dim = 24
max_length = 256

# Model Definition with LSTM
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Set the training parameters
model_lstm.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model_lstm.summary()'''

"import tensorflow as tf\n\n# Parameters\nembedding_dim = 16\nlstm_dim = 32\ndense_dim = 24\nmax_length = 256\n\n# Model Definition with LSTM\nmodel_lstm = tf.keras.Sequential([\n    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),\n    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim)),\n    tf.keras.layers.Dense(dense_dim, activation='relu'),\n    tf.keras.layers.Dense(1, activation='sigmoid')\n])\n\n# Set the training parameters\nmodel_lstm.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])\n\n# Print the model summary\nmodel_lstm.summary()"

In [ ]:
X_train.shape

(199963,)

In [ ]:
trans = vectorizer.fit_transform(newX_test)

In [ ]:
type(trans)

scipy.sparse.csr.csr_matrix

In [ ]:
newX_test.shape

(40391,)

In [ ]:
testrans = vectorizer.fit_transform(newX_test)

In [ ]:
testranstoarr = testrans.toarray()

In [ ]:
len(testranstoarr[0])

622

In [ ]:
transtoarr = trans.toarray()

In [ ]:
for x in testranstoarr[23000]:
  if(x!=0.0):
    print(x)

0.15321831400361574
0.446755205515474
0.27442780353862956
0.3361462324980235
0.6539836110114808
0.40116637714226255


In [ ]:
len(transtoarr[0])

622

In [ ]:
len(transtoarr)

40391

In [ ]:
len(newy_test)

40391

In [ ]:
#so 156242 rows and 7102 columns

In [ ]:
trans.shape[0]

40391

In [ ]:
#tdf = pd.DataFrame(trans[0].T.todense(), index=vectorizer.get_feature_names(), columns=["TF-IDF"])

In [ ]:
#tdf = tdf.sort_values('TF-IDF', ascending=False)

In [ ]:
#tdf.shape

In [ ]:
#from gensim import models

In [ ]:
#custom_model = models.Word2Vec(X_train, min_count=1,size=300,workers=4)

In [ ]:
#STREAM LEARNING

In [ ]:
!pip install river

     |████████████████████████████████| 2.0 MB 5.3 MB/s 


In [ ]:
from river import stream
from river import compose
from river import tree
from river import preprocessing
from river import metrics
#from sklearn import metrics
from river import multiclass
from river import linear_model

In [ ]:
print(transtoarr)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.4085556  0.04351107 0.04768649 ... 0.         0.         0.        ]
 ...
 [0.25506708 0.09007203 0.09871555 ... 0.         0.         0.        ]
 [1.         0.         0.         ... 0.         0.         0.        ]
 [0.15647884 0.         0.         ... 0.         0.         0.        ]]


In [ ]:
stream_dataset = stream.iter_array(transtoarr, newy_test)

In [ ]:
stream_model = compose.Pipeline(
    preprocessing.StandardScaler(),
    #multiclass.OneVsRestClassifier(linear_model.PAClassifier())
    tree.HoeffdingAdaptiveTreeClassifier(grace_period=100,split_confidence=1e-5,leaf_prediction='nb',nb_threshold=10,seed=0)
)

In [ ]:
#stream_model = tree.HoeffdingAdaptiveTreeClassifier(grace_period=100,split_confidence=1e-5,leaf_prediction='nb',nb_threshold=10,seed=0)

In [ ]:
print(stream_model)

StandardScaler | HoeffdingAdaptiveTreeClassifier


In [ ]:
batch_times = {}
batch_scores = {}
online_times = {}
online_scores = {}
online_sumtimes = {}

In [ ]:
import time

In [ ]:
metric = metrics.Accuracy()
cm = metrics.ConfusionMatrix()
onlinetrain_no=0
onlinesum=0
start_time=time.time()
y_pred = [0]


for xi, yi in stream_dataset:
  preds = stream_model.predict_one(xi)
  y_pred.append(preds)
  tic=time.time()
  stream_model = stream_model.learn_one(xi, yi)
  iter_time = time.time()-tic
  online_times[onlinetrain_no] = iter_time
  onlinesum+=iter_time
  online_sumtimes[onlinetrain_no]=onlinesum
  metric = metric.update(yi, preds)
  cm = cm.update(yi, preds)
  #metric = metrics.accuracy_score(yi, preds)
  online_scores[onlinetrain_no]= metric.get()
  #the online train no has to be added by one while printing because, it has been started by 0 for the sake of the array
  #there are 7267 total entries in this dataset, finally to calculate the average the value of total entries which is at 7266(because starts from 0) must be added by 1
  print("INFO] acc update {} - {}".format(onlinetrain_no+1,metric)) 
  print("INFO] training time {} - {} secs\n".format(onlinetrain_no+1,iter_time))
  onlinetrain_no+=1

tot_time = time.time() - start_time
onlinetrain_no+=1

In [ ]:
print("\n\n[INFO] final accuracy {}- {}".format(onlinetrain_no,metric))
print("[INFO] total training time - {}".format(onlinesum))
print("[INFO] average training time - {}".format(onlinesum/onlinetrain_no))



[INFO] final accuracy 40392- Accuracy: 90.60%
[INFO] total training time - 120.17104315757751
[INFO] average training time - 0.0029751199038814003


In [ ]:
del(y_pred[1])
print(y_pred)

[0, 3, 3, 3, 1, 0, 0, 0, 1, 0, 0, 3, 0, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 1, 0, 3, 3, 1, 0, 3, 1, 0, 1, 2, 3, 3, 1, 2, 1, 1, 1, 0, 3, 3, 3, 1, 1, 0, 3, 3, 1, 0, 1, 1, 3, 3, 3, 1, 3, 1, 1, 0, 1, 1, 3, 0, 0, 1, 1, 0, 1, 3, 1, 0, 3, 0, 3, 3, 3, 1, 3, 2, 3, 3, 3, 0, 3, 2, 0, 0, 0, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 3, 0, 1, 2, 1, 1, 1, 1, 0, 3, 3, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 0, 3, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 0, 2, 3, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 3, 0, 1, 0, 1, 1, 3, 1, 1, 1, 1, 1, 0, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 3, 1, 0, 1, 3, 3, 1, 1, 0, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 0, 3, 3, 1, 1, 1, 3, 1, 1, 3, 1, 3, 1, 1, 1, 1, 3, 3, 1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 3, 1, 2, 1, 1, 1, 1, 1, 0, 1, 3, 3, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 1, 3, 1, 3, 3, 1, 1, 1, 0, 3, 3, 1, 1, 1, 

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
sk_cm = confusion_matrix(newy_test, y_pred)
print(sk_cm)
print(accuracy_score(newy_test, y_pred))
cl_report = classification_report(newy_test,y_pred)
print(cl_report)

[[15047  1341   248    76]
 [ 1103 13527   258    67]
 [  215   347   238    15]
 [   61    52    14  7782]]
0.9059939095343021
              precision    recall  f1-score   support

           0       0.92      0.90      0.91     16712
           1       0.89      0.90      0.90     14955
           2       0.31      0.29      0.30       815
           3       0.98      0.98      0.98      7909

    accuracy                           0.91     40391
   macro avg       0.77      0.77      0.77     40391
weighted avg       0.91      0.91      0.91     40391

